In [1]:
from pkdtools import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import rebound

from mpl_toolkits.mplot3d import Axes3D  # not always necessary but good for older versions
import imageio
import os
import seaborn as sns
import pandas as pd


G = 6.67430e-11  # m^3 kg^-1 s^-2
M_center = 5.683e26  # kg
yr = 3.15576e7  # seconds
M_sun = 1.989e30  # kg

au = 1.496e11  # m
rho = 0.4e3

r = 1.666    # 0.25 - 0.75

R_roche = 1.05 * (M_center/rho)**(1/3)

Sigma = 400  # kg/m^2
tau = 3 * Sigma / (4 * rho * r)


orbit_radius = 1.3e8  # m
orbit_speed = np.sqrt(G * M_center / orbit_radius)
omega_orb = np.sqrt(G * M_center / (orbit_radius)**3)
period_orb = 2 * np.pi / omega_orb
escape_speed = np.sqrt(2 * G * 4/3 * rho * r**3 / r)
unit_speed = omega_orb * r


lambda_crit = 4 * np.pi**2 * G * Sigma / omega_orb**2
Lx = 200
Ly = 200
Lz = 1

theoretical_collision_speed = Lz / orbit_radius * orbit_speed

mean_free_path = Lz / tau
shear_speed = 3/2 * omega_orb * (mean_free_path)
theoretical_collision_speed = Lz / orbit_radius * orbit_speed

r_h = orbit_radius * (2 * 4*np.pi/3 * rho * r**3 / 3 / M_center)**(1/3)
if r_h < r:
    v_rel = 2 * r * omega_orb
else:
    v_rel = np.sqrt(G * 4*np.pi/3 * rho * r**3 / r)
v_rel = max(escape_speed, shear_speed)
Q = v_rel * omega_orb / np.pi / G / Sigma

print('Oribit Period (h): ', period_orb/3600)
print('timestep (h): ',1e-07 * yr/2/np.pi/3600)
print('orbital radius: ', orbit_radius, orbit_radius/ R_roche)
print('orbital speed (m/s): ', orbit_speed)
print('lambda crit (m): ', lambda_crit)
print('Surface density (kg/m^2): ', Sigma)
print('Scale height: ', Lz / orbit_radius)
print('Unit speed (m/s): ', unit_speed)
print('Escape speed: ', escape_speed/unit_speed)
print('Shear speed: ', shear_speed/unit_speed)

print('Theoretical collision speed: ', theoretical_collision_speed/unit_speed)
print('dimensions: ', Lx, Ly, Lz)
print('Toomre parameter Q: ', Q)

Oribit Period (h):  13.283138660566872
timestep (h):  0.00013951522311435544
orbital radius:  130000000.0 1.1013233107783533
orbital speed (m/s):  17081.264342502916
lambda crit (m):  61.04809676352157
Surface density (kg/m^2):  400
Scale height:  7.692307692307693e-09
Unit speed (m/s):  0.00021890297226622964
Escape speed:  2.030675654914225
Shear speed:  2.0000000000000004
Theoretical collision speed:  0.6002400960384155
dimensions:  200 200 1
Toomre parameter Q:  0.6963912319512788


In [2]:
13/1e-4, Lx/au, orbit_radius/au

(130000.0, 1.3368983957219252e-09, 0.0008689839572192514)

In [3]:
0.00000067 * au, orbit_radius

(100232.0, 130000000.0)

In [4]:
# examine the code

volume = Lx * Ly * Lz 
N_total = 2837
sigma = np.pi * r**2
optical_depth = N_total * sigma / volume * Lz

print('Theoretical optical depth: ~', optical_depth)
print('Input optical depth: ', tau)

Theoretical optical depth: ~ 0.61844233510968
Input optical depth:  0.45018007202881155


In [ ]:
Particles = ss_in('ss.0000010000', units = 'pkd')

In [17]:
# plot the particles

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import imageio.v2 as imageio

def plot_particles(filename, output_image='particles.png'):
    pkd_particles = ss_in(filename, units = 'mks')
    p_position = np.zeros((len(pkd_particles), 3))
    p_radius = np.zeros(len(pkd_particles))



    for i in range(len(pkd_particles)):
        p_position[i] = np.array([pkd_particles[i].x, pkd_particles[i].y, pkd_particles[i].z])
        p_radius[i] = pkd_particles[i].R

    # print(np.sum(np.pi * p_radius**2) / 200**2)
    # print(Sigma * 200**2, np.sum(4*np.pi/3 * rho * p_radius**3) )


    plt.figure(figsize=[8, 8])
    ax = plt.axes([0.1, 0.1, 0.8, 0.8], xlim=(-100, 100), ylim=(-100, 100))

    points_whole_ax = 8 * 0.8 * 72    # 1 point = dpi / 72 pixels
    plot_range = 2 * 100 # x_max - x_min
    # points_radius = 2 * radius / range * points_whole_ax

    ax.set_ylabel("radial coordinate [m]")
    ax.set_xlabel("azimuthal coordinate [m]")

    ax.scatter(p_position[:, 1], p_position[:, 0], s = (2 * p_radius / plot_range * points_whole_ax)**2,facecolor='darkgray', edgecolor='black')
    
    # plt.show()

    plt.savefig(output_image)
    plt.close()



def generate_gif(filenames, output_path='gif/particles.gif'):
    images = []
    for idx, fname in enumerate(filenames):
        print('processing frame', idx, 'of', len(filenames))
        image_file = f"frame_{idx:03d}.png"
        plot_particles(fname, image_file)
        images.append(imageio.imread(image_file))
        os.remove(image_file)

    imageio.mimsave(output_path, images)

    # # Clean up temp files
    # for image_file in images:
    #     os.remove(image_file)


# plot the particles
# plot_particles('ss.0000130000')
# plot_particles('initcond.ss')



# # generate the gif
start_index = 0
end_index = 130000
filenames = ['ss.' + str(i).zfill(10) for i in range(start_index, end_index, 1000)]

generate_gif(filenames, output_path='pkd.gif')



processing frame 0 of 130
processing frame 1 of 130
processing frame 2 of 130
processing frame 3 of 130
processing frame 4 of 130
processing frame 5 of 130
processing frame 6 of 130
processing frame 7 of 130
processing frame 8 of 130
processing frame 9 of 130
processing frame 10 of 130
processing frame 11 of 130
processing frame 12 of 130
processing frame 13 of 130
processing frame 14 of 130
processing frame 15 of 130
processing frame 16 of 130
processing frame 17 of 130
processing frame 18 of 130
processing frame 19 of 130
processing frame 20 of 130
processing frame 21 of 130
processing frame 22 of 130
processing frame 23 of 130
processing frame 24 of 130
processing frame 25 of 130
processing frame 26 of 130
processing frame 27 of 130
processing frame 28 of 130
processing frame 29 of 130
processing frame 30 of 130
processing frame 31 of 130
processing frame 32 of 130
processing frame 33 of 130
processing frame 34 of 130
processing frame 35 of 130
processing frame 36 of 130
processing 

In [ ]:


# plot gif of spins
def plot_spin_gif(filenames, plot = 'spin rate', output_path='gif/spin.gif'):
    images = []

    for idx, filename in enumerate(filenames):
        print(f"Processing frame {idx} of {len(filenames)}")
        Particles = ss_in(filename, units='mks')
        p_spin = np.zeros((len(Particles), 3))
        p_spin_rate = np.zeros(len(Particles))
        obliquity = np.zeros(len(Particles))
        for i in range(len(Particles)):
            p_spin[i] = np.array([Particles[i].wx, Particles[i].wy, Particles[i].wz + omega_orb ])
            p_spin_rate[i] = np.linalg.norm(p_spin[i])
            obliquity[i] = np.arccos(p_spin[i, 2] / np.linalg.norm(p_spin[i])) * 180/np.pi

        
        
        if plot == 'spin rate':
            fig = plt.figure()
            plt.title(f'Frame {idx}')
            plt.hist(p_spin_rate/omega_orb, bins=30)
            plt.xlabel('Spin Rate')
            plt.ylabel('Count')

        elif plot == 'obliquity':
            fig = plt.figure()
            plt.title(f'Frame {idx}')
            plt.hist(obliquity, bins=30)
            plt.xlabel('Obliquity')
            plt.ylabel('Count')

        elif plot == 'distribution':
                # Create the figure
            fig = plt.figure(figsize=(8, 8))
            gs = gridspec.GridSpec(4, 4)

            # Main scatter plot
            ax_main = fig.add_subplot(gs[1:4, 0:3])
            ax_main.scatter(obliquity, p_spin_rate/omega_orb,  alpha=0.5)
            ax_main.set_xlim([0, 180])
            ax_main.set_ylim([0, 10])
            ax_main.set_xlabel('Obliquity')
            ax_main.set_ylabel('Spin Rate')
            ax_main.set_title(f'Frame {idx}')

            # Histogram on the top
            ax_top = fig.add_subplot(gs[0, 0:3], sharex=ax_main)
            ax_top.hist(obliquity, bins=50, alpha=0.6)
            ax_top.axis('off')  # Optional: hide axes if you want a cleaner look

            # Histogram on the right
            ax_right = fig.add_subplot(gs[1:4, 3], sharey=ax_main)
            ax_right.hist(p_spin_rate/omega_orb, bins=50, orientation='horizontal', alpha=0.6)
            ax_right.axis('off')  # Optional

            plt.tight_layout()

            

        # Save the frame to a temporary image
        temp_filename = f'_frame_{idx}.png'
        plt.savefig(temp_filename, dpi=100)
        plt.close(fig)

        images.append(imageio.imread(temp_filename))
        os.remove(f'_frame_{idx}.png')

    # Save the images as a GIF
    imageio.mimsave(output_path, images)

    # Clean up temp files
    # for idx in range(len(filenames)):
    #     os.remove(f'_frame_{idx}.png')

    print(f"GIF saved to {output_path}")

start_index = 10000
end_index = 1000000
filenames = ['ss.' + str(i).zfill(10) for i in range(start_index, end_index, 10000)]

print(filenames)


plot_spin_gif(filenames, plot = 'distribution')

In [ ]:
p = ss_in('ss.0000500000', units='mks')


obliquity = np.zeros(len(p))
spin_rate = np.zeros(len(p))

for i in range(len(p)):
    wx = p[i].wx
    wy = p[i].wy
    wz = p[i].wz
    w = np.array([wx, wy, wz])
    spin_rate[i] = np.linalg.norm(w)  # magnitude of the spin vector
    if spin_rate[i] == 0:
        obliquity[i] = np.nan
    else:
        obliquity[i] = np.arccos(w[2] / np.linalg.norm(w))  # angle with respect to z-axis

# plt.hist(np.cos(obliquity), bins=30, alpha=0.5)
# plt.hist(spin_rate, bins=30, alpha=0.5)
print(obliquity)

In [ ]:
# plot a Maxwellian distribution of the spin rate
def maxwellian(v, v0):
    return (v**2 / (v0**3 * np.sqrt(np.pi/2))) * np.exp(-v**2 / (2 * v0**2))

def rayleigh(x, sigma):
    return (x / sigma**2) * np.exp(-x**2 / (2 * sigma**2))

def plot_maxwellian(v0, max_val, bins=30):
    v = np.linspace(0, max_val, 1000)
    y = maxwellian(v, v0)

    fig, ax = plt.subplots()
    ax.plot(v, y)
    ax.set_xlabel('Spin Rate')
    ax.set_ylabel('Probability Density')
    ax.set_title(f'Maxwellian Distribution with v0={v0}')
    plt.vlines(np.sqrt(2), 0, 0.5, color='r', linestyle='--', label='v0')

    plt.show()


def plot_spin(filename, plot = 'spin rate'):
    Particles = ss_in(filename, units='mks')
    p_spin = np.zeros((len(Particles), 3))
    p_spin_rate = np.zeros(len(Particles))
    obliquity = np.zeros(len(Particles))
    for i in range(len(Particles)):
        p_spin[i] = np.array([Particles[i].wx, Particles[i].wy, Particles[i].wz + omega_orb])
        p_spin_rate[i] = np.linalg.norm(p_spin[i])
        obliquity[i] = np.arccos(p_spin[i, 2] / np.linalg.norm(p_spin[i])) * 180/np.pi

    
    if plot == 'spin rate':
        fig, ax = plt.subplots()
        ax.hist(p_spin_rate/omega_orb, bins=30, density=True)
        ax.set_xlabel('Spin Rate')
        ax.set_ylabel('Count')

        # plot maxwellian distribution
        v0 = np.sqrt(2)
        # v0 = np.mean(p_spin_rate/omega_orb)/np.sqrt(2)
        v = np.linspace(0, 10, 1000)
        y = rayleigh(v, v0)
        # ax_maxwell = fig.add_subplot(gs[0, 3])
        ax.plot(v, y)
        ax.set_xlabel('Spin Rate')
        ax.set_ylabel('Probability Density')

    elif plot == 'obliquity':
        fig, ax = plt.subplots()
        ax.hist(obliquity, bins=30, density=True)
        ax.set_xlabel('Obliquity')
        ax.set_ylabel('Count')

    elif plot == 'distribution':
        # Create the figure
        fig = plt.figure(figsize=(8, 8))
        gs = gridspec.GridSpec(4, 4)

        # Main scatter plot
        ax_main = fig.add_subplot(gs[1:4, 0:3])
        ax_main.scatter(obliquity, p_spin_rate/omega_orb, alpha=0.5)
        ax_main.set_xlim([0, 180])
        ax_main.set_ylim([0, 10])
        ax_main.set_xlabel('Spin Rate')
        ax_main.set_ylabel('Obliquity')
        # ax_main.set_title(f'Frame {idx}')

        # Histogram on the top
        ax_top = fig.add_subplot(gs[0, 0:3], sharex=ax_main)
        ax_top.hist(p_spin_rate/omega_orb, bins=50, alpha=0.6)
        ax_top.axis('off')  # Optional: hide axes if you want a cleaner look

        # Histogram on the right
        ax_right = fig.add_subplot(gs[1:4, 3], sharey=ax_main)
        ax_right.hist(obliquity, bins=50, orientation='horizontal', alpha=0.6)
        ax_right.axis('off')  # Optional

        


        plt.tight_layout()


    plt.show()

plot_spin('ss.0000500000', plot = 'obliquity')



# plot_maxwellian(1, 10, bins=30)


In [ ]:
(1.5e14)**(1/3) * 0.7816

In [50]:
# Rayleigh distribution
def rayleigh(x, sigma):
    return (x / sigma**2) * np.exp(-x**2 / (2 * sigma**2))

def plot_rayleigh(sigma, max_val, bins=30):
    x = np.linspace(0, max_val, 1000)
    y = rayleigh(x, sigma)

    fig, ax = plt.subplots()
    ax.plot(x, y)
    ax.set_xlabel('Spin Rate')
    ax.set_ylabel('Probability Density')
    ax.set_title(f'Rayleigh Distribution with sigma={sigma}')
    plt.vlines(np.sqrt(2), 0, 0.5, color='r', linestyle='--', label='v0')

    plt.show()

In [ ]:
import rebound
import numpy as np
sim = rebound.Simulation()

OMEGA = 0.00013143527     # [1/s]

sim.ri_sei.OMEGA = OMEGA
surface_density = 400.    # kg/m^2
particle_density = 400.   # kg/m^3
sim.G = 6.67428e-11       # N m^2 / kg^2
sim.dt = 1e-3*2.*np.pi/OMEGA
sim.softening = 0.2       # [m]
boxsize = 200.            # [m]

tau = surface_density / particle_density / 1


Q = np.sqrt(sim.G * 4/3 * np.pi * particle_density * 3**3) * OMEGA / np.pi / sim.G / surface_density

sim.configure_box(boxsize)
sim.N_ghost_x = 2
sim.N_ghost_y = 2
sim.integrator = "sei"
sim.boundary   = "shear"
sim.gravity    = "tree"
sim.collision  = "tree"
sim.collision_resolve = "hardsphere"
def cor_bridges(r, v):
        eps = 0.32*pow(abs(v)*100.,-0.234)
        if eps>1.:
            eps=1.
        if eps<0.:
            eps=0.
        return eps
sim.coefficient_of_restitution = cor_bridges

def powerlaw(slope, min_v, max_v):
    y = np.random.uniform()
    pow_max = pow(max_v, slope+1.)
    pow_min = pow(min_v, slope+1.)
    return pow((pow_max-pow_min)*y + pow_min, 1./(slope+1.))

total_mass = 0.
while total_mass < surface_density*(boxsize**2):
    radius = powerlaw(slope=-3, min_v=1, max_v=4)  # [m]    
    mass = particle_density*4./3.*np.pi*(radius**3)
    x = np.random.uniform(low=-boxsize/2., high=boxsize/2.)
    sim.add(
        m=mass,
        r=radius,
        x=x,
        y=np.random.uniform(low=-boxsize/2., high=boxsize/2.),
        z=np.random.normal(),
        vx = 0.,
        vy = -3./2.*x*OMEGA, 
        vz = 0.)
    total_mass += mass

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
def plotParticles(sim):
    fig = plt.figure(figsize=(8,8))
    ax = plt.subplot(111,aspect='equal')
    ax.set_ylabel("radial coordinate [m]")
    ax.set_xlabel("azimuthal coordinate [m]")
    ax.set_ylim(-boxsize/2.,boxsize/2.)
    ax.set_xlim(-boxsize/2.,boxsize/2.)

    for i, p in enumerate(sim.particles):
        circ = patches.Circle((p.y, p.x), p.r, facecolor='darkgray', edgecolor='black')
        ax.add_patch(circ)

plotParticles(sim)
print(Q)

In [ ]:
raidus = np.zeros(len(sim.particles))
for i in range(len(sim.particles)):
    raidus[i] = sim.particles[i].r
np.sum(np.pi * radius**2) / 200**2, surface_density * 200**2, np.sum(4*np.pi/3 * particle_density * radius**3) 

In [ ]:
sim.integrate(2.*np.pi/OMEGA)
plotParticles(sim)